In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" an

In [ ]:
import torch
import pandas as pd

In [127]:
data = pd.read_csv("passengers.csv")

In [128]:
data.head()

,desc,result
0,Gender: Male\nAge: 72\nDeparture City: Kazan\n...,False
1,Gender: Male\nAge: 21\nDeparture City: Saint P...,True
2,Gender: Female\nAge: 23\nDeparture City: Yekat...,False
3,Gender: Male\nAge: 42\nDeparture City: Nadym\n...,True
4,Gender: Female\nAge: 65\nDeparture City: Novy ...,False


In [129]:
from sklearn.model_selection import train_test_split

X = data['desc'].to_list()
Y = [int(y) for y in data['result']]
train_texts, val_texts, train_labels, val_labels = train_test_split(X, Y, test_size=.2)
val_texts[0], val_labels[0], train_labels.count(1), train_labels.count(0)

('Gender: Female\nAge: 27\nDeparture City: Krasnoyarsk\nDestination City: Magadan\nClass: First\nPrice: 67176\nRefundable: No\nPassenger Type: Business\nFlight Frequency: 3\nBaggage Availability: No\nPurchase Method: Travel agency\nBooking Group: Alone\nTime Before Flight: 78\nFlight Duration: 0\nWeather at Departure: Snowfall\nTransfer: Yes\nDelay Between Flights: No\nDay of the Week: Wednesday\nPublic Holiday: Yes\nSocial Status: Working',
 0,
 3067,
 4933)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True)
val_encodings = tokenizer(val_texts, truncation=True)


class UserDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_dataset = UserDataset(train_encodings, train_labels)
val_dataset = UserDataset(val_encodings, val_labels)

Encoding(num_tokens=421, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [126]:
train_encodings[0].tokens

['[CLS]',
 'gender',
 ':',
 'male',
 'age',
 ':',
 '70',
 'departure',
 'city',
 ':',
 'st',
 '##av',
 '##rop',
 '##ol',
 'destination',
 'city',
 ':',
 'che',
 '##ly',
 '##abi',
 '##nsk',
 'class',
 ':',
 'first',
 'price',
 ':',
 '53',
 '##6',
 '##7',
 '##1',
 'ref',
 '##unda',
 '##ble',
 ':',
 'yes',
 'passenger',
 'type',
 ':',
 'regular',
 'flight',
 'frequency',
 ':',
 '7',
 'baggage',
 'availability',
 ':',
 'no',
 'purchase',
 'method',
 ':',
 'online',
 'booking',
 'group',
 ':',
 'alone',
 'time',
 'before',
 'flight',
 ':',
 '8',
 'flight',
 'duration',
 ':',
 '2',
 'weather',
 'at',
 'departure',
 ':',
 'clear',
 'transfer',
 ':',
 'no',
 'day',
 'of',
 'the',
 'week',
 ':',
 'thursday',
 'public',
 'holiday',
 ':',
 'yes',
 'social',
 'status',
 ':',
 'working',
 '[SEP]']

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "Pass", 1: "Miss"}
label2id = {"Pass": 0, "Miss": 1}

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
  "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments(
    output_dir="flight_miss_prediction",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.670100,0.675812,0.593500


TrainOutput(global_step=500, training_loss=0.6701361083984375, metrics={'train_runtime': 365.1382, 'train_samples_per_second': 21.91, 'train_steps_per_second': 1.369, 'total_flos': 903034396244160.0, 'train_loss': 0.6701361083984375, 'epoch': 1.0})

In [ ]:
model2 = AutoModelForSequenceClassification.from_pretrained("flight_miss_prediction/checkpoint-500")

In [125]:
one_indexes = [i for i in range(len(train_labels)) if train_labels[i]]
text = train_texts[one_indexes[100]]
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    logits = model2(**inputs).logits
    print(logits[0].softmax(dim=0))
    predicted_class_id = logits.argmax().item()
    print(model.config.id2label[predicted_class_id])


tensor([0.3864, 0.6136])
Miss
